In [1]:
import uproot
import awkward as ak
from coffea import nanoevents
from coffea.nanoevents.methods.base import NanoEventsArray
from coffea.analysis_tools import Weights, PackedSelection
from coffea.nanoevents.methods import nanoaod
from coffea.nanoevents.methods import vector
from coffea.lookup_tools.dense_lookup import dense_lookup

ak.behavior.update(vector.behavior)

import pickle, json, gzip
import numpy as np

from typing import Optional, List, Dict
from copy import copy

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from tqdm import tqdm

import os

# import corrections
import correctionlib

# import utils

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
d_PDGID = 1
u_PDGID = 2
s_PDGID = 3
c_PDGID = 4
b_PDGID = 5
g_PDGID = 21
TOP_PDGID = 6

ELE_PDGID = 11
vELE_PDGID = 12
MU_PDGID = 13
vMU_PDGID = 14
TAU_PDGID = 15
vTAU_PDGID = 16

Z_PDGID = 23
W_PDGID = 24
HIGGS_PDGID = 25
Y_PDGID = 35

b_PDGIDS = [511, 521, 523]

GRAV_PDGID = 39

GEN_FLAGS = ["fromHardProcess", "isLastCopy"]

In [ ]:
plot_dir = "../../../plots/ScaleFactors/Nov23"
_ = os.system(f"mkdir -p {plot_dir}")

In [9]:
P4 = {
    "eta": "Eta",
    "phi": "Phi",
    "mass": "Mass",
    "pt": "Pt",
}


PAD_VAL = -99999


skim_vars = {
    "FatJet": {
        **P4,
        "msoftdrop": "Msd",
        "particleNetMD_QCD": "ParticleNetMD_QCD",
        "particleNetMD_Xbb": "ParticleNetMD_Xbb",
        "particleNet_H4qvsQCD": "ParticleNet_Th4q",
        "particleNet_mass": "ParticleNetMass",
    },
    "GenHiggs": P4,
    "other": {"MET_pt": "MET_pt"},
}


def pad_val(
    arr: ak.Array,
    target: int,
    value: float = PAD_VAL,
    axis: int = 0,
    to_numpy: bool = True,
    clip: bool = True,
):
    """
    pads awkward array up to ``target`` index along axis ``axis`` with value ``value``,
    optionally converts to numpy array
    """
    ret = ak.fill_none(ak.pad_none(arr, target, axis=axis, clip=clip), value, axis=axis)
    return ret.to_numpy() if to_numpy else ret


def add_selection(
    name: str,
    sel: np.ndarray,
    selection: PackedSelection,
    cutflow: dict,
    isData: bool,
    genWeights: ak.Array = None,
):
    """adds selection to PackedSelection object and the cutflow dictionary"""
    if isinstance(sel, ak.Array):
        sel = sel.to_numpy()

    selection.add(name, sel.astype(bool))
    cutflow[name] = (
        np.sum(selection.all(*selection.names))
        if isData
        # add up genWeights for MC
        else np.sum(genWeights[selection.all(*selection.names)])
    )

In [4]:
events = nanoevents.NanoEventsFactory.from_root(
    # "../../../../data/2017_UL_nano/NMSSM_XToYH_MX1000_MY400_HTo2bYTo2W_hadronicDecay/nano_mc2017_101.root",
    "/eos/uscms//store/user/lpcpfnano/rkansal/v2_3/2016APV/XHY/NMSSM_XToYHTo2W2BTo4Q2B_MX-3000_MY-190_TuneCP5_13TeV-madgraph-pythia8/NMSSM_XToYHTo2W2BTo4Q2B_MX-3000_MY-190/230323_173705/0000/nano_mc2016pre_13.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

/uscms_data/d3/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:75: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1813 branches) at 0x7fc25ce1fc10>, taking first instance
  warnings.warn(
/uscms_data/d3/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:75: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1813 branches) at 0x7fc25ce1fc10>, taking first instance
  warnings.warn(
/uscms_data/d3/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:75: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1813 branches) at 0x7fc25ce1fc10>, taking first instance
  warnings.warn(
/uscms_data/d3/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:75: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Eve

In [7]:
isData = False
signGenWeights = None if isData else np.sign(events["genWeight"])
n_events = len(events) if isData else int(np.sum(signGenWeights))
selection = PackedSelection()

cutflow = {}
cutflow["all"] = len(events)

In [5]:
events

<NanoEventsArray [<event 1:15:13064>, ... ] type='3732 * event'>

In [13]:
skim_vars = P4
fatjets = events.FatJet

higgs = events.GenPart[
    (abs(events.GenPart.pdgId) == HIGGS_PDGID) * events.GenPart.hasFlags(GEN_FLAGS)
]
GenHiggsVars = {f"GenHiggs{key}": higgs[var].to_numpy() for (var, key) in skim_vars.items()}
is_bb = abs(higgs.children.pdgId) == b_PDGID
has_bb = ak.sum(ak.flatten(is_bb, axis=2), axis=1) == 2

bb = ak.flatten(higgs.children[is_bb], axis=2)
GenbbVars = {f"Genbb{key}": pad_val(bb[var], 2, axis=1) for (var, key) in skim_vars.items()}

# gen Y and kids
Ys = events.GenPart[(abs(events.GenPart.pdgId) == Y_PDGID) * events.GenPart.hasFlags(GEN_FLAGS)]
GenYVars = {f"GenY{key}": Ys[var].to_numpy() for (var, key) in skim_vars.items()}
is_VV = (abs(Ys.children.pdgId) == W_PDGID) + (abs(Ys.children.pdgId) == Z_PDGID)
has_VV = ak.sum(ak.flatten(is_VV, axis=2), axis=1) == 2

add_selection("has_bbVV", has_bb * has_VV, selection, cutflow, False, signGenWeights)

VV = ak.flatten(Ys.children[is_VV], axis=2)
GenVVVars = {f"GenVV{key}": VV[var][:, :2].to_numpy() for (var, key) in skim_vars.items()}

VV_children = VV.children
quarks = abs(VV_children.pdgId) <= b_PDGID
all_q = ak.all(ak.all(quarks, axis=2), axis=1)
add_selection("all_q", all_q, selection, cutflow, False, signGenWeights)

V_has_2q = ak.count(VV_children.pdgId, axis=2) == 2
has_4q = ak.values_astype(ak.prod(V_has_2q, axis=1), bool)
add_selection("has_4q", has_4q, selection, cutflow, False, signGenWeights)

Gen4qVars = {
    f"Gen4q{key}": ak.to_numpy(
        ak.fill_none(
            ak.pad_none(ak.pad_none(VV_children[var], 2, axis=1, clip=True), 2, axis=2, clip=True),
            PAD_VAL,
        )
    )
    for (var, key) in skim_vars.items()
}

# fatjet gen matching
Hbb = ak.pad_none(higgs, 1, axis=1, clip=True)[:, 0]
HVV = ak.pad_none(Ys, 1, axis=1, clip=True)[:, 0]

bbdr = fatjets[:, :2].delta_r(Hbb)
vvdr = fatjets[:, :2].delta_r(HVV)

match_dR = 0.8
Hbb_match = bbdr <= match_dR
HVV_match = vvdr <= match_dR

# overlap removal - in the case where fatjet is matched to both, match it only to the closest Higgs
Hbb_match = (Hbb_match * ~HVV_match) + (bbdr <= vvdr) * (Hbb_match * HVV_match)
HVV_match = (HVV_match * ~Hbb_match) + (bbdr > vvdr) * (Hbb_match * HVV_match)

VVJets = ak.pad_none(fatjets[HVV_match], 1, axis=1)[:, 0]
quarkdrs = ak.flatten(VVJets.delta_r(VV_children), axis=2)
num_prongs = ak.sum(quarkdrs < match_dR, axis=1)

GenMatchingVars = {
    "ak8FatJetHbb": pad_val(Hbb_match, 2, axis=1),
    "ak8FatJetHVV": pad_val(HVV_match, 2, axis=1),
    "ak8FatJetHVVNumProngs": ak.fill_none(num_prongs, PAD_VAL).to_numpy(),
}

genbb, gen4q = bb, ak.flatten(VV_children, axis=2)

In [14]:
gen4q

<GenParticleArray [[GenParticle, ... GenParticle]] type='3732 * var * ?genParticle'>

In [15]:
fatjet_idx = 0
ak8_pfcands = events.FatJetPFCands
ak8_pfcands = ak8_pfcands[ak8_pfcands.jetIdx == fatjet_idx]
pfcands0 = events.PFCands[ak8_pfcands.pFCandsIdx]

fatjet_idx = 1
ak8_pfcands = events.FatJetPFCands
ak8_pfcands = ak8_pfcands[ak8_pfcands.jetIdx == fatjet_idx]
pfcands1 = events.PFCands[ak8_pfcands.pFCandsIdx]

In [21]:
sel = np.prod(
    pad_val(
        (fatjets.pt > 250) * (np.abs(fatjets.eta) < 2.4) * (fatjets.particleNet_mass >= 50),
        2,
        False,
        axis=1,
    ),
    axis=1,
)

In [23]:
np.min(ak.count(pfcands1[sel].pt, axis=1))

49

In [ ]:
fatjet_idx = 0
ak8_pfcands = events.FatJetPFCands
ak8_pfcands = ak8_pfcands[ak8_pfcands.jetIdx == fatjet_idx]
pfcands0 = events.PFCands[ak8_pfcands.pFCandsIdx]

fatjet_idx = 1
ak8_pfcands = events.FatJetPFCands
ak8_pfcands = ak8_pfcands[ak8_pfcands.jetIdx == fatjet_idx]
pfcands1 = events.PFCands[ak8_pfcands.pFCandsIdx]